在本节中，我们将着眼一个关于拓宽机器人活动范围的问题：如何走出迷宫？如果你有一个打扫房间的Roomba吸尘机器人，你可能会想根据你在本章所学知识对它重新编程了。我们要做的是帮助海龟在虚拟的迷宫寻找出路。迷宫问题可以追溯到希腊神话中忒修斯进入迷宫猎杀人身牛头怪的故事。忒修斯用了一个线球以使自己在杀掉怪物后能够找到出口。在我们的问题中，我们假设我们的海龟落入迷宫的中央，需要找到出路。请看图1.思考如何走出迷宫。为了简化问题，我们假定这个迷宫被分成正方形。每块小方格要么开放要么被不可通过的墙壁占据。海龟只能通过迷宫中开放的部分，如果海龟遇到墙就需要尝试不同方向。海龟需要一个系统的步骤以找到正确的方式走出迷宫，下面是具体的步骤:
- 在初始位置尝试向北走一步，以此为开始递归程序。
- 北面走不通的情况下则向南尝试，其后开始递归。
- 南面走不通的情况下则向西尝试，其后开始递归。
- 如果北面、南面、西面都走不通，则从东面开始并递归程序。
- 如果四个方向都走不出，则被困在迷宫中，以失败告终。
![图1.已解决的迷宫问题](pic/4.11.1.png)
<center>图1.已解决的迷宫问题</center>

尽管看起来简单但仍有几处细节需要说明。假设我们第一步迈向北，按照程序下一步也将是向北。但是如果北边是墙壁，我们将按程序的下一步尝试向南。不幸的是走向南后，将回到起点。按照递归将陷入无尽的循环。因此，我们需要一个记住走过的路径的方法。想象我们有一包面包屑可以沿路撒下，如果我们准备向某个方向前进时发现路上已经有了面包屑，我们应该立刻退回并尝试下一个方向。在算法的代码中我们将看到回退一步与返回递归调用同样方便。
对于所有的递归算法我们都要找到基本情况。在前面几段描述中，已经设想了几种情况。在此算法中，有四种基本情况需要考虑:
1. 海龟碰到“墙壁”，方格被占用无法通行。
2. 海龟发现表示此方格已访问过，为避免陷入循环不在此位置继续寻找。
3. 海龟碰到位于边缘的通道，即找到迷宫出口。
4. 海龟在四个方向上探索都失败。

为了让程序运行，我们需要用一种方法来表示迷宫。为了使之更为生动，我们将用海龟模块来绘
制并探索迷宫，这样我们可以看到算法的动态效果。迷宫对象将提供以下几种方法供我们在写算法
时使用:
- \__init__ 用以读取迷宫数据，初始化迷宫内部，并找到海龟初始位置。
- draw_maze 用以在屏幕上绘制迷宫。
- update_position 用以更新迷宫内的状态及在窗口中改变海龟位置。
- is_exit 用以判断当前位置是否为出口。

迷宫类包含索引操作符\[ \]以使算法能够方便地存取任一方格的状态。

让我们来看一下被称为searchFrom的搜索代码。代码如下。注意此函数包括三个参数：迷宫对象、起始行、起始列。由于递归中每一次调用在逻辑上都要重新开始搜索，这一点十分重要。

In [3]:
def searchFrom(maze, startRow, startColumn):
    maze.updatePosition(startRow, startColumn)
    #  Check for base cases:
    #  1. We have run into an obstacle, return false
    if maze[startRow][startColumn] == OBSTACLE:
        return False
    #  2. We have found a square that has already been explored
    if maze[startRow][startColumn] == TRIED:
        return False
    # 3. Success, an outside edge not occupied by an obstacle
    if maze.isExit(startRow,startColumn):
        maze.updatePosition(startRow, startColumn, PART_OF_PATH)
        return True
    maze.updatePosition(startRow, startColumn, TRIED)

    # Otherwise, use logical short circuiting to try each
    # direction in turn (if needed)
    found = searchFrom(maze, startRow-1, startColumn) or \
            searchFrom(maze, startRow+1, startColumn) or \
            searchFrom(maze, startRow, startColumn-1) or \
            searchFrom(maze, startRow, startColumn+1)
    if found:
        maze.updatePosition(startRow, startColumn, PART_OF_PATH)
    else:
        maze.updatePosition(startRow, startColumn, DEAD_END)
    return found

在算法中可以看到代码所做的第一件事是调用updatePosition(第二行)。这一步是为了将算法可视化，如此，海龟在迷宫中的移动就可以被观察到。接下来算法查验了四种基本情况的前三种：海龟是否碰壁(第五行)？海龟是否回到标记过的地方(第八行)？海龟是否找到了出口(第十一行)？如果这些情况无一符合，递归搜索继续。

你会发现，在递归过程中有四次递归调用searchFrom。很难估算这些递归将被调用多少次，因为它们彼此联系。如果searchFrom第一次调用返回true，则后三个不需要调用。可以将此理解为向(当前行数-1，当前列数)前进一步(即向北)是走出迷宫的一个步骤。如果向北走并非可行之路则将尝试向南的递归调用。如果南边失败，则尝试向西，最后是向东。如果全部四个递归调用都返回false，则说明是死胡同。这时应重新编译程序，对不同的调用顺序进行尝试。

迷宫类的代码如下所示。\__init__方法接受一个文件名称作为其唯一参数。此文件是使用“+”字符作为墙壁围出空心正方形空间，并用字母“S”来表示起始位置的迷宫文本文件。下面即为一个迷宫的例子。
```python
++++++++++++++++++++++
+   +   ++ ++     +
+ +   +       +++ + ++
+ + +  ++  ++++   + ++
+++ ++++++    +++ +  +
+          ++  ++    +
+++++ ++++++   +++++ +
+     +   +++++++  + +
+ +++++++      S +   +
+                + +++
++++++++++++++++++ +++
```

迷宫的内部则数据项为字符列表的列表表示。所述maze_list实例变量的每一行也是列表。该二级列表中每一个位置由上述一种字符填充。数据文件的内部表示如下:

```python
[ ['+','+','+','+',...,'+','+','+','+','+','+','+'],
  ['+',' ',' ',' ',...,' ',' ',' ','+',' ',' ',' '],
  ['+',' ','+',' ',...,'+','+',' ','+',' ','+','+'],
  ['+',' ','+',' ',...,' ',' ',' ','+',' ','+','+'],
  ['+','+','+',' ',...,'+','+',' ','+',' ',' ','+'],
  ['+',' ',' ',' ',...,'+','+',' ',' ',' ',' ','+'],
  ['+','+','+','+',...,'+','+','+','+','+',' ','+'],
  ['+',' ',' ',' ',...,'+','+',' ',' ','+',' ','+'],
  ['+',' ','+','+',...,' ',' ','+',' ',' ',' ','+'],
  ['+',' ',' ',' ',...,' ',' ','+',' ','+','+','+'],
  ['+','+','+','+',...,'+','+','+',' ','+','+','+']]
```

update_position方法，如下所示，采用内部显示观察海龟是否已经碰壁。它还用“.”或“ - ”更新内部表示来指示乌龟以访问过的位置与死胡同。此外，该方法使用两个辅助方法move_turtle和drop_bread_crumb以更新屏幕上的图。最后，is_exit方法用当前位置验证出口。当海龟到达0行、0列、最右或最末行时满足出口条件。

In [4]:
class Maze:
    def __init__(self,mazeFileName):
        rowsInMaze = 0
        columnsInMaze = 0
        self.mazelist = []
        mazeFile = open(mazeFileName,'r')
        rowsInMaze = 0
        for line in mazeFile:
            rowList = []
            col = 0
            for ch in line[:-1]:
                rowList.append(ch)
                if ch == 'S':
                    self.startRow = rowsInMaze
                    self.startCol = col
                col = col + 1
            rowsInMaze = rowsInMaze + 1
            self.mazelist.append(rowList)
            columnsInMaze = len(rowList)

        self.rowsInMaze = rowsInMaze
        self.columnsInMaze = columnsInMaze
        self.xTranslate = -columnsInMaze/2
        self.yTranslate = rowsInMaze/2
        self.t = Turtle(shape='turtle')
        setup(width=600,height=600)
        setworldcoordinates(-(columnsInMaze-1)/2-.5,
                            -(rowsInMaze-1)/2-.5,
                            (columnsInMaze-1)/2+.5,
                            (rowsInMaze-1)/2+.5)

In [5]:
def drawMaze(self):
    for y in range(self.rowsInMaze):
        for x in range(self.columnsInMaze):
            if self.mazelist[y][x] == OBSTACLE:
                self.drawCenteredBox(x+self.xTranslate,
                                     -y+self.yTranslate,
                                     'tan')
    self.t.color('black','blue')

def drawCenteredBox(self,x,y,color):
    tracer(0)
    self.t.up()
    self.t.goto(x-.5,y-.5)
    self.t.color('black',color)
    self.t.setheading(90)
    self.t.down()
    self.t.begin_fill()
    for i in range(4):
        self.t.forward(1)
        self.t.right(90)
    self.t.end_fill()
    update()
    tracer(1)

def moveTurtle(self,x,y):
    self.t.up()
    self.t.setheading(self.t.towards(x+self.xTranslate,
                                     -y+self.yTranslate))
    self.t.goto(x+self.xTranslate,-y+self.yTranslate)

def dropBreadcrumb(self,color):
    self.t.dot(color)

def updatePosition(self,row,col,val=None):
    if val:
        self.mazelist[row][col] = val
    self.moveTurtle(col,row)

    if val == PART_OF_PATH:
        color = 'green'
    elif val == OBSTACLE:
        color = 'red'
    elif val == TRIED:
        color = 'black'
    elif val == DEAD_END:
        color = 'red'
    else:
        color = None

    if color:
        self.dropBreadcrumb(color)

In [6]:
def isExit(self,row,col):
     return (row == 0 or
             row == self.rowsInMaze-1 or
             col == 0 or
             col == self.columnsInMaze-1 )

def __getitem__(self,idx):
     return self.mazelist[idx]

完整的程序展示在最后。程序使用以下所示存有迷宫的数据文件maze2.txt。应注意本例中出口与海龟位置距离很近故十分简单。
```python
++++++++++++++++++++++
+   +   ++ ++        +
      +     ++++++++++
+ +    ++  ++++ +++ ++
+ +   + + ++    +++  +
+          ++  ++  + +
+++++ + +      ++  + +
+++++ +++  + +  ++   +
+          + + S+ +  +
+++++ +  + + +     + +
++++++++++++++++++++++
```

In [8]:
import turtle

PART_OF_PATH = 'O'
TRIED = '.'
OBSTACLE = '+'
DEAD_END = '-'

class Maze:
    def __init__(self,mazeFileName):
        rowsInMaze = 0
        columnsInMaze = 0
        self.mazelist = []
        mazeFile = open(mazeFileName,'r')
        rowsInMaze = 0
        for line in mazeFile:
            rowList = []
            col = 0
            for ch in line[:-1]:
                rowList.append(ch)
                if ch == 'S':
                    self.startRow = rowsInMaze
                    self.startCol = col
                col = col + 1
            rowsInMaze = rowsInMaze + 1
            self.mazelist.append(rowList)
            columnsInMaze = len(rowList)

        self.rowsInMaze = rowsInMaze
        self.columnsInMaze = columnsInMaze
        self.xTranslate = -columnsInMaze/2
        self.yTranslate = rowsInMaze/2
        self.t = turtle.Turtle()
        self.t.shape('turtle')
        self.wn = turtle.Screen()
        self.wn.setworldcoordinates(-(columnsInMaze-1)/2-.5,-(rowsInMaze-1)/2-.5,(columnsInMaze-1)/2+.5,(rowsInMaze-1)/2+.5)

    def drawMaze(self):
        self.t.speed(10)
        self.wn.tracer(0)
        for y in range(self.rowsInMaze):
            for x in range(self.columnsInMaze):
                if self.mazelist[y][x] == OBSTACLE:
                    self.drawCenteredBox(x+self.xTranslate,-y+self.yTranslate,'orange')
        self.t.color('black')
        self.t.fillcolor('blue')
        self.wn.update()
        self.wn.tracer(1)

    def drawCenteredBox(self,x,y,color):
        self.t.up()
        self.t.goto(x-.5,y-.5)
        self.t.color(color)
        self.t.fillcolor(color)
        self.t.setheading(90)
        self.t.down()
        self.t.begin_fill()
        for i in range(4):
            self.t.forward(1)
            self.t.right(90)
        self.t.end_fill()

    def moveTurtle(self,x,y):
        self.t.up()
        self.t.setheading(self.t.towards(x+self.xTranslate,-y+self.yTranslate))
        self.t.goto(x+self.xTranslate,-y+self.yTranslate)

    def dropBreadcrumb(self,color):
        self.t.dot(10,color)

    def updatePosition(self,row,col,val=None):
        if val:
            self.mazelist[row][col] = val
        self.moveTurtle(col,row)

        if val == PART_OF_PATH:
            color = 'green'
        elif val == OBSTACLE:
            color = 'red'
        elif val == TRIED:
            color = 'black'
        elif val == DEAD_END:
            color = 'red'
        else:
            color = None

        if color:
            self.dropBreadcrumb(color)

    def isExit(self,row,col):
        return (row == 0 or
                row == self.rowsInMaze-1 or
                col == 0 or
                col == self.columnsInMaze-1 )

    def __getitem__(self,idx):
        return self.mazelist[idx]


def searchFrom(maze, startRow, startColumn):
    # try each of four directions from this point until we find a way out.
    # base Case return values:
    #  1. We have run into an obstacle, return false
    maze.updatePosition(startRow, startColumn)
    if maze[startRow][startColumn] == OBSTACLE :
        return False
    #  2. We have found a square that has already been explored
    if maze[startRow][startColumn] == TRIED or maze[startRow][startColumn] == DEAD_END:
        return False
    # 3. We have found an outside edge not occupied by an obstacle
    if maze.isExit(startRow,startColumn):
        maze.updatePosition(startRow, startColumn, PART_OF_PATH)
        return True
    maze.updatePosition(startRow, startColumn, TRIED)
    # Otherwise, use logical short circuiting to try each direction
    # in turn (if needed)
    found = searchFrom(maze, startRow-1, startColumn) or \
            searchFrom(maze, startRow+1, startColumn) or \
            searchFrom(maze, startRow, startColumn-1) or \
            searchFrom(maze, startRow, startColumn+1)
    if found:
        maze.updatePosition(startRow, startColumn, PART_OF_PATH)
    else:
        maze.updatePosition(startRow, startColumn, DEAD_END)
    return found


myMaze = Maze('maze2.txt')
myMaze.drawMaze()
myMaze.updatePosition(myMaze.startRow,myMaze.startCol)

searchFrom(myMaze, myMaze.startRow, myMaze.startCol)

True